## DV2599 Assignment 1
Group 8


***
Viktor Fransson

vifr22@student.bth.se

***

Tobias Gustafsson

togu22@student.bth.se
***

#### Initialize

In [74]:
# Import packages
import pandas as pd
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from imblearn.over_sampling import SMOTE

# Load dataset
df = pd.read_csv('winequality-white.csv', delimiter=";")

#### 1. Inspect the dataset

In [75]:
# Describe
# df.describe()
# df.info()

# Calculate ratios of quality classes
class_counts = df['quality'].value_counts().sort_index()

print("Quality Class Ratios:")
for class_label, count in class_counts.items():
    ratio = count / len(df)
    print(f"Class {class_label}: {ratio:.2%}")


# Calculate ratio of feature values per quality class
# Get all feature columns
feature_columns = [col for col in df.columns if col != 'quality']

# Create copy of df to change and keep original intact
df_discretisized = df.copy()

for feature in feature_columns:
    df_discretisized[feature] = pd.qcut(df[feature], q=3, labels=['low', 'medium', 'high'])

# Analyze each target class
print("\nTarget Class Feature Analysis:")
for class_label in sorted(df_discretisized['quality'].unique()):
    print(f"\nClass {class_label}:")
    
    # Subset df for the current class
    class_df = df_discretisized[df_discretisized['quality'] == class_label]
    
    # Find the most frequent feature value for each feature
    for feature in feature_columns:
        top_values = class_df[feature].value_counts().head(2)
        print(f"  {feature}:")
        for value, count in top_values.items():
            print(f"    {value}: {count/len(class_df):.1%}")

Quality Class Ratios:
Class 3: 0.41%
Class 4: 3.33%
Class 5: 29.75%
Class 6: 44.88%
Class 7: 17.97%
Class 8: 3.57%
Class 9: 0.10%

Target Class Feature Analysis:

Class 3:
  fixed acidity:
    high: 50.0%
    low: 25.0%
  volatile acidity:
    high: 45.0%
    medium: 30.0%
  citric acid:
    high: 40.0%
    low: 30.0%
  residual sugar:
    high: 40.0%
    low: 35.0%
  chlorides:
    low: 40.0%
    high: 35.0%
  free sulfur dioxide:
    low: 45.0%
    high: 30.0%
  total sulfur dioxide:
    high: 55.0%
    low: 35.0%
  density:
    high: 40.0%
    medium: 35.0%
  pH:
    low: 35.0%
    medium: 35.0%
  sulphates:
    low: 45.0%
    high: 30.0%
  alcohol:
    medium: 45.0%
    low: 35.0%

Class 4:
  fixed acidity:
    high: 39.9%
    low: 34.4%
  volatile acidity:
    high: 57.7%
    medium: 27.0%
  citric acid:
    low: 48.5%
    high: 31.9%
  residual sugar:
    low: 48.5%
    medium: 30.1%
  chlorides:
    high: 42.9%
    medium: 30.1%
  free sulfur dioxide:
    low: 69.3%
    medium: 

#### 2. Split into train and test sets

In [76]:
# Set quality as target
y = df['quality']
x = df.drop('quality', axis=1)

# Split data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

#### 3. Scaling

In [77]:
# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform train set
x_train_scaled = scaler.fit_transform(x_train)

# Transform test set
x_test_scaled = scaler.transform(x_test)

# Turn back into pandas dataframes
x_train_scaled = pd.DataFrame(x_train_scaled, columns=x_train.columns, index=x_train.index)
x_test_scaled = pd.DataFrame(x_test_scaled, columns=x_test.columns, index=x_test.index)

# Describe
x_train_scaled.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
count,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000
mean,0.287261,0.214523,0.201288,0.088246,0.109277,0.115671,0.299277,0.132780,0.429415,0.312526,0.408538
std,0.082285,0.108663,0.072556,0.078555,0.065549,0.059169,0.098282,0.058437,0.138741,0.131879,0.199264
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.233010,0.140541,0.162651,0.016871,0.080119,0.073171,0.229698,0.088490,0.330275,0.220930,0.241935
50%,0.281553,0.194595,0.192771,0.069018,0.100890,0.111498,0.290023,0.127048,0.422018,0.290698,0.387097
75%,0.330097,0.259459,0.234940,0.142638,0.121662,0.149826,0.364269,0.173318,0.513761,0.383721,0.548387
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#### 4. Repeated k-Fold Cross Validation

In [78]:
# Initialize Repeated k-Fold Cross Validator
rkf = RepeatedKFold(n_splits=3, n_repeats=10)

# Random forest classifier and Logistic regression
classifiers = {
    "Random Forest" : RandomForestClassifier(),
    "Decision Tree" : DecisionTreeClassifier()
}

# Perform cross-validation
results = {}
for name, clf in classifiers.items():
    cv_scores = cross_val_score(clf, x_train_scaled, y_train, cv=rkf, scoring="accuracy")

    results[name] = {
        "average" : round(cv_scores.mean(), 4),
        "standard deviation" : round(cv_scores.std(), 4)
    }

results

{'Random Forest': {'average': 0.6441, 'standard deviation': 0.0128},
 'Decision Tree': {'average': 0.5502, 'standard deviation': 0.0122}}

#### 5. Final model

In [79]:
# Random forest was the best classifier
rf_model = RandomForestClassifier()

rf_model.fit(x_train_scaled, y_train)

RandomForestClassifier()

#### 6. Performance on test set

In [80]:
# Run model on test set
y_pred = rf_model.predict(x_test_scaled)

# Check accuracy
accuracy = accuracy_score(y_pred, y_test)
accuracy

0.6959183673469388

#### 7. Balance scaled train set

In [81]:
# Apply SMOTE to train set to balance
smote = SMOTE(k_neighbors=3) # k_neigbors < smallest class size (4)

x_train_bal, y_train_bal = smote.fit_resample(x_train_scaled, y_train)

# Check class distribution
class_counts = pd.Series(y_train).value_counts()
# print(class_counts)
balanced_class_counts = pd.Series(y_train_bal).value_counts()
# print(balanced_class_counts)

# Calculate class ratios
class_ratios = class_counts / len(y_train)
bal_class_ratios = balanced_class_counts / len(y_train)
print("\nClass ratios:")
print(class_ratios)
print("\nBalanced:")
print(bal_class_ratios)


Class ratios:
quality
6    0.451761
5    0.295304
7    0.179173
8    0.035733
4    0.032925
3    0.004084
9    0.001021
Name: count, dtype: float64

Balanced:
quality
5    0.451761
8    0.451761
6    0.451761
7    0.451761
9    0.451761
4    0.451761
3    0.451761
Name: count, dtype: float64


#### 8. Validation and fitting on balanced set

In [82]:
# Perform cross-validation (may take a while)
results = {}
for name, clf in classifiers.items():
    cv_scores = cross_val_score(clf, x_train_bal, y_train_bal, cv=rkf, scoring="accuracy")

    results[name] = {
        "average" : round(cv_scores.mean(), 4),
        "standard deviation" : round(cv_scores.std(), 4)
    }

results

{'Random Forest': {'average': 0.892, 'standard deviation': 0.0051},
 'Decision Tree': {'average': 0.815, 'standard deviation': 0.0058}}

In [83]:
# Random forest was the best classifier again
rf_model_bal = RandomForestClassifier()

rf_model_bal.fit(x_train_scaled, y_train)

RandomForestClassifier()

#### 9. Performance of model trained on balanced set

In [84]:
# Run model on test set
y_pred = rf_model_bal.predict(x_test_scaled)

# Check for improvement
accuracy_bal = accuracy_score(y_pred, y_test)
print("Accuracy improvement:", round(accuracy_bal - accuracy, 3))

Accuracy improvement: 0.012
